In [22]:
from typing import List
import itertools
import json
from copy import deepcopy
import itertools
import json
import os
import shutil
import sys
from typing import List
import json
import os
import sys
from distutils.dir_util import copy_tree
from typing import Dict

In [5]:
def prepare_scenarios(
    distributions: List[str],
    values_of_change: List[float]
):
    lists_to_product = []
    for dist in distributions:
        lists_to_product.append(
            list(itertools.product([dist], values_of_change))
        )

    all_combinations = list(itertools.product(*lists_to_product))

    return all_combinations


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [23]:
dists = ['a', 'b', 'c']
values = [0, 0.1, 0.2, 0.3]

scenarios = prepare_scenarios(dists, values)

In [24]:
len(scenarios)

64

In [25]:
chunks = chunkIt(scenarios, 10)

In [26]:
len(chunks)

11

In [27]:
s = 0
for c in chunks:
    print(len(c))
    s += len(c)
print()
print(s)

6
6
7
6
7
6
6
7
6
6
1

64


In [28]:
def change_dist(
    dist: Dict[str, Dict[str, float]],
    value: float
):

    if value != 0:

        if value < 0:
            value = abs(value)
            reverse = False
        else:
            reverse = True

        for dist_key in dist.keys():
            d = dist[dist_key]
            prev_key = None
            for k in sorted(d.keys(), reverse=reverse):
                if prev_key:
                    change_value = value * d[k]
                    d[prev_key] = d[prev_key] + change_value
                    d[k] = d[k] - change_value
                prev_key = k
            dist[dist_key] = d

    return dist

In [29]:
dist = {
    'a': {
        '1': 0.2,
        '2': 0.3,
        '3': 0.5
    },
    'b': {
        '1': 0.1,
        '2': 0.1,
        '3': 0.3,
        '4': 0.25,
        '5': 0.25
    }
}

In [31]:
change_dist(deepcopy(dist), 0.1)

{'a': {'1': 0.18, '2': 0.29000000000000004, '3': 0.53},
 'b': {'1': 0.09, '2': 0.1, '3': 0.28, '4': 0.255, '5': 0.275}}

In [33]:
change_dist(deepcopy(dist), -0.1)

{'a': {'1': 0.23, '2': 0.32, '3': 0.45},
 'b': {'1': 0.11000000000000001,
  '2': 0.12,
  '3': 0.29500000000000004,
  '4': 0.25,
  '5': 0.225}}

In [35]:
def get_output_folder_postfix(scenario):
    postfix = ''

    for [dist, value] in scenario:
        postfix += '_'
        postfix += dist.split('/')[-1]
        postfix += '_'
        postfix += str(value).replace('.', '_')

    return postfix

In [38]:
get_output_folder_postfix([["decision_tree/pub_trans_comfort_dist-up", 0.1], ["decision_tree/pub_trans_punctuality_dist-up", 0.0], ["decision_tree/household_cars_dist-down", 0.5]])

'_pub_trans_comfort_dist-up_0_1_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_5'